<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Network" data-toc-modified-id="Load-Network-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Network</a></span></li><li><span><a href="#Project" data-toc-modified-id="Project-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Project</a></span></li><li><span><a href="#Generate-Images" data-toc-modified-id="Generate-Images-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Generate Images</a></span></li></ul></div>

Project/embed real images into StyleGANv2 latent space.

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
from datetime import datetime
from tqdm import tqdm

# ffmpeg installation location, for creating videos
plt.rcParams['animation.ffmpeg_path'] = str(Path.home() / "Documents/dev_tools/ffmpeg-20190623-ffa64a4-win64-static/bin/ffmpeg.exe")

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

%load_ext autoreload
%autoreload 2

# StyleGAN Utils
from stylegan_utils import load_network, gen_image_fun, synth_image_fun, create_video

# StyleGAN2 Repo
sys.path.append(os.path.join(os.pardir, 'stylegan2encoder'))

import dnnlib
import dataset_tool
import run_projector
import projector
import training.dataset
import training.misc

# Data Science Utils
sys.path.append(os.path.join(os.pardir, 'data-science-learning'))

from ds_utils import generative_utils

In [ ]:
res_dir = Path.home() / 'Documents/generated_data/stylegan'

# Load Network

In [ ]:
MODELS_DIR = Path("C:/Users/User/Documents/models/stylegan2")
MODEL_NAME = 'original_ffhq'
SNAPSHOT_NAME = 'stylegan2-ffhq-config-f'

Gs, Gs_kwargs, noise_vars = load_network(str(MODELS_DIR / MODEL_NAME / SNAPSHOT_NAME) + '.pkl')

Z_SIZE = Gs.input_shape[1:][0]
IMG_SIZE = Gs.output_shape[2:]
IMG_SIZE

# Project

In [ ]:
def project_images(images_dir, tfrecord_dir, data_dir, num_steps, num_snapshots):
    # setup projector
    print('Setting up projector')
    proj = projector.Projector(num_steps=num_steps)
    proj.set_network(Gs)
    
    # generate tfrecords
    nb_images = dataset_tool.create_from_images(str(tfrecord_dir), str(images_dir), True)

    # loading images from tfrecords
    dataset_obj = training.dataset.load_dataset(data_dir=str(data_dir), tfrecord_dir=tfrecord_dir, 
                                                max_label_size=0, verbose=True, repeat=False, shuffle_mb=0)
    assert dataset_obj.shape == Gs.output_shape[1:]
    
    # project all loaded images
    print('=======================')
    for image_idx in tqdm(range(nb_images)):
        print(f'Projecting image {image_idx}/{nb_images}')
        
        images, _labels = dataset_obj.get_minibatch_np(1)
        images = training.misc.adjust_dynamic_range(images, [0, 255], [-1, 1])
        
        run_path = data_dir / f'out_{image_idx}'
        run_path.mkdir()
        run_projector.project_image(proj, targets=images, 
                                    png_prefix=dnnlib.make_run_dir_path(str(run_path / 'image_')), 
                                    num_snapshots=num_snapshots)

In [ ]:
data_dir = res_dir / 'projection' / MODEL_NAME / SNAPSHOT_NAME / datetime.now().strftime("%Y%m%d_%H%M%S") # where the projections results will be saved
images_dir = Path.home() / 'Documents/generated_data/face_extract' / 'tmp_portraits'

tfrecord_dir = data_dir / 'tfrecords'
project_images(images_dir=images_dir, tfrecord_dir=tfrecord_dir, data_dir=data_dir, 
               num_steps=2000, num_snapshots=50)

In [ ]:
create_video(data_dir / 'out_7', 
             res_dir / 'projection' / 'out_{}.mp4'.format('test'))

# Generate Images

In [ ]:
# network args
Gs_kwargs.truncation_psi = 0.1

In [ ]:
target_latents = np.random.rand(1, Z_SIZE)

In [ ]:
img = gen_image_fun(Gs, target_latents, noise_vars, Gs_kwargs)
plt.imshow(img)

In [ ]:
img = synth_image_fun(Gs, target_latents[np.newaxis,:,:], randomize_noise=True)
plt.imshow(img)